### Carregando os dados

In [103]:
import numpy as np
import pandas as pd

In [104]:
def load_matches(path):
    matches = pd.read_json(path_or_buf=path)
    # as informações dos times de cada partida estão em um dicionário dentro da coluna 'teamsData', então vamos separar essas informações
    team_matches = []
    for i in range(len(matches)):
        match = pd.DataFrame(matches.loc[i, 'teamsData']).T
        match['matchId'] = matches.loc[i, 'wyId']
        team_matches.append(match)
    team_matches = pd.concat(team_matches).reset_index(drop=True)

    return team_matches

In [105]:
def load_players(path):
    players = pd.read_json(path_or_buf=path)
    players['player_name'] = players['firstName'] + ' ' + players['lastName']
    players = players[['wyId', 'player_name']].rename(columns={'wyId': 'player_id'})

    return players

In [106]:
def load_events(path):
    events = pd.read_json(path_or_buf=path)
    # pré processamento em colunas da tabela de eventos para facilitar a conversão p/ SPADL
    events = events.rename(columns={
        'id': 'event_id',
        'eventId': 'type_id',
        'subEventId': 'subtype_id',
        'teamId': 'team_id',
        'playerId': 'player_id',
        'matchId': 'game_id'
    })
    events['milliseconds'] = events['eventSec'] * 1000
    events['period_id'] = events['matchPeriod'].replace({'1H': 1, '2H': 2})

    return events

In [107]:
def load_minutes_played_per_game(path):
    minutes = pd.read_json(path_or_buf=path)
    minutes = minutes.rename(columns={
        'playerId': 'player_id',
        'matchId': 'game_id',
        'teamId': 'team_id',
        'minutesPlayed': 'minutes_played'
    })
    minutes = minutes.drop(['shortName', 'teamName', 'red_card'], axis=1)

    return minutes

In [108]:
leagues = ['England', 'Spain', 'Italy', 'France', 'Germany']
events = {}
matches = {}
minutes = {}
for league in leagues:
    path = r'C:\UFMG\CDAF\matches\matches_{}.json'.format(league)
    matches[league] = load_matches(path)
    path = r'C:\UFMG\CDAF\dados\events_{}.json'.format(league)
    events[league] = load_events(path)
    path = r'C:\UFMG\CDAF\dados\minutes_played_per_game_{}.json'.format(league)
    minutes[league] = load_minutes_played_per_game(path)

In [109]:
path = r'C:\UFMG\CDAF\players\players.json'
players = load_players(path)
players['player_name'] = players['player_name'].str.decode('unicode-escape')

### SPADL

In [110]:
%pip install socceraction

Note: you may need to restart the kernel to use updated packages.


In [111]:
from tqdm import tqdm
import socceraction.spadl as spd

In [112]:
def spadl_transform(events, matches):
    spadl = []
    game_ids = events.game_id.unique().tolist()
    for g in tqdm(game_ids):
        match_events = events.loc[events.game_id == g]
        match_home_id = matches.loc[(matches.matchId == g) & (matches.side == 'home'), 'teamId'].values[0]
        match_actions = spd.wyscout.convert_to_actions(events=match_events, home_team_id=match_home_id)
        match_actions = spd.play_left_to_right(actions=match_actions, home_team_id=match_home_id)
        match_actions = spd.add_names(match_actions)
        spadl.append(match_actions)
    spadl = pd.concat(spadl).reset_index(drop=True)

    return spadl

In [113]:
spadl = {}
for league in leagues:
    spadl[league] = spadl_transform(events=events[league], matches=matches[league])

  0%|          | 0/380 [00:00<?, ?it/s]

100%|██████████| 306/306 [03:47<00:00,  1.34it/s]


### Features

In [114]:
from socceraction.vaep import features as ft

In [115]:
def features_transform(spadl):
    spadl.loc[spadl.result_id.isin([2, 3]), ['result_id']] = 0
    spadl.loc[spadl.result_name.isin(['offside', 'owngoal']), ['result_name']] = 'fail'

    xfns = [
        ft.actiontype_onehot,
        ft.bodypart_onehot,
        ft.result_onehot,
        ft.goalscore,
        ft.startlocation,
        ft.endlocation,
        ft.team,
        ft.time,
        ft.time_delta
    ]

    features = []
    for game in tqdm(np.unique(spadl.game_id).tolist()):
        match_actions = spadl.loc[spadl.game_id == game].reset_index(drop=True)
        match_states = ft.gamestates(actions=match_actions)
        match_feats = pd.concat([fn(match_states) for fn in xfns], axis=1)
        features.append(match_feats)
    features = pd.concat(features).reset_index(drop=True)

    return features

In [116]:
features = {}
for league in ['England', 'Spain', 'Italy', 'Germany', 'France']:
    features[league] = features_transform(spadl[league])

  0%|          | 0/380 [00:00<?, ?it/s]

100%|██████████| 380/380 [00:18<00:00, 20.21it/s]


### Labels

In [117]:
import socceraction.vaep.labels as lab

In [118]:
def labels_transform(spadl):
    yfns = [lab.scores, lab.concedes]

    labels = []
    for game in tqdm(np.unique(spadl.game_id).tolist()):
        match_actions = spadl.loc[spadl.game_id == game].reset_index(drop=True)
        labels.append(pd.concat([fn(actions=match_actions) for fn in yfns], axis=1))

    labels = pd.concat(labels).reset_index(drop=True)

    return labels

In [119]:
labels = {}
for league in ['England', 'Spain', 'Italy', 'Germany', 'France']:
    labels[league] = labels_transform(spadl[league])

  0%|          | 0/380 [00:00<?, ?it/s]

100%|██████████| 380/380 [00:24<00:00, 15.36it/s]


In [120]:
labels['England']['scores'].sum()

7553

In [121]:
labels['England']['concedes'].sum()

2313

### Training Model

In [122]:
%pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [123]:
import xgboost as xgb
import sklearn.metrics as mt

In [124]:
def train_vaep(X_train, y_train, X_test, y_test):
    models = {}
    for m in ['scores', 'concedes']:
        models[m] = xgb.XGBClassifier(random_state=0, n_estimators=50, max_depth=3)

        print('training ' + m + ' model')
        models[m].fit(X_train, y_train[m])

        p = sum(y_train[m]) / len(y_train[m])
        base = [p] * len(y_train[m])
        y_train_pred = models[m].predict_proba(X_train)[:, 1]
        train_brier = mt.brier_score_loss(y_train[m], y_train_pred) / mt.brier_score_loss(y_train[m], base)
        print(m + ' Train NBS: ' + str(train_brier))
        print()

        p = sum(y_test[m]) / len(y_test[m])
        base = [p] * len(y_test[m])
        y_test_pred = models[m].predict_proba(X_test)[:, 1]
        test_brier = mt.brier_score_loss(y_test[m], y_test_pred) / mt.brier_score_loss(y_test[m], base)
        print(m + ' Test NBS: ' + str(test_brier))
        print()

        print('----------------------------------------')

    return models

In [126]:
models = train_vaep(X_train=features['England'], y_train=labels['England'], X_test=features['Spain'], y_test=labels['Spain'])

training scores model
scores Train NBS: 0.8452154331682665

scores Test NBS: 0.8503669232527757

----------------------------------------
training concedes model
concedes Train NBS: 0.9644632156409795

concedes Test NBS: 0.9745272571693011

----------------------------------------


### Predictions

In [150]:
def generate_predictions(features, models):
    preds = {}
    for m in ['scores', 'concedes']:
        preds[m] = models[m].predict_proba(features)[:, 1]
    preds = pd.DataFrame(preds)

    return preds

In [151]:
preds = {}
preds['Spain'] = generate_predictions(features=features['Spain'], models=models)
preds['England'] = generate_predictions(features=features['England'], models=models)
preds['Germany'] = generate_predictions(features=features['Germany'], models=models)
preds['France'] = generate_predictions(features=features['France'], models=models)
preds['Italy'] = generate_predictions(features=features['Italy'], models=models)

### Action Values

In [152]:
import socceraction.vaep.formula as fm

In [153]:
def calculate_action_values(spadl, predictions):
    action_values = fm.value(actions=spadl, Pscores=predictions['scores'], Pconcedes=predictions['concedes'])
    action_values = pd.concat([
        spadl[['player_id', 'original_event_id', 'action_id', 'game_id', 'start_x', 'start_y', 'end_x', 'end_y', 'type_name', 'result_name']],
        predictions.rename(columns={'scores': 'Pscores', 'concedes': 'Pconcedes'}),
        action_values
    ], axis=1)

    return action_values

In [154]:
action_values = {}
action_values['Spain'] = calculate_action_values(spadl=spadl['Spain'], predictions=preds['Spain'])
action_values['England'] = calculate_action_values(spadl=spadl['England'], predictions=preds['England'])
action_values['Germany'] = calculate_action_values(spadl=spadl['Germany'], predictions=preds['Germany'])
action_values['France'] = calculate_action_values(spadl=spadl['France'], predictions=preds['France'])
action_values['Italy'] = calculate_action_values(spadl=spadl['Italy'], predictions=preds['Italy'])

### Player Ratings

In [155]:
def calculate_minutes_per_season(minutes_per_game):
    minutes_per_season = minutes_per_game.groupby('player_id', as_index=False)['minutes_played'].sum()

    return minutes_per_season

In [156]:
minutes_per_season = {}
minutes_per_season['Spain'] = calculate_minutes_per_season(minutes['Spain'])
minutes_per_season['England'] = calculate_minutes_per_season(minutes['England'])
minutes_per_season['Germany'] = calculate_minutes_per_season(minutes['Germany'])
minutes_per_season['France'] = calculate_minutes_per_season(minutes['France'])
minutes_per_season['Italy'] = calculate_minutes_per_season(minutes['Italy'])

In [157]:
def calculate_player_ratings(action_values, minutes_per_season, players):
    player_ratings = action_values.groupby(by='player_id', as_index=False).agg({'vaep_value': 'sum'}).rename(columns={'vaep_value': 'vaep_total'})
    player_ratings = player_ratings.merge(minutes_per_season, on=['player_id'], how='left')
    player_ratings['vaep_p90'] = player_ratings['vaep_total'] / player_ratings['minutes_played'] * 90
    player_ratings = player_ratings[player_ratings['minutes_played'] >= 600].sort_values(by='vaep_p90', ascending=False).reset_index(drop=True)
    player_ratings = player_ratings.merge(players, on=['player_id'], how='left')
    player_ratings = player_ratings[['player_id', 'player_name', 'minutes_played', 'vaep_total', 'vaep_p90']]

    return player_ratings

In [158]:
minutes_per_season['Spain']

,player_id,minutes_played
0,33,93
1,99,103
2,151,1379
3,254,3230
4,786,83
...,...,...
552,519496,65
553,520163,124
554,545811,24
555,551398,63


In [159]:
player_ratings = {}
player_ratings['Spain'] = calculate_player_ratings(action_values=action_values['Spain'], minutes_per_season=minutes_per_season['Spain'], players=players)
player_ratings['England'] = calculate_player_ratings(action_values=action_values['England'], minutes_per_season=minutes_per_season['England'], players=players)
player_ratings['Germany'] = calculate_player_ratings(action_values=action_values['Germany'], minutes_per_season=minutes_per_season['Germany'], players=players)
player_ratings['France'] = calculate_player_ratings(action_values=action_values['France'], minutes_per_season=minutes_per_season['France'], players=players)
player_ratings['Italy'] = calculate_player_ratings(action_values=action_values['Italy'], minutes_per_season=minutes_per_season['Italy'], players=players)

In [160]:
player_ratings['Spain']

,player_id,player_name,minutes_played,vaep_total,vaep_p90
0,3359,Lionel Andrés Messi Cuccittini,3108.0,35.891377,1.039326
1,8278,Gareth Frank Bale,1850.0,14.323647,0.696826
2,3802,Philippe Coutinho Correia,1329.0,10.036555,0.679676
3,3322,Cristiano Ronaldo dos Santos Aveiro,2355.0,17.279105,0.660348
4,3682,Antoine Griezmann,2591.0,18.149570,0.630437
...,...,...,...,...,...
386,105616,Brown Ideye Aide,764.0,0.059302,0.006986
387,5911,Sandro Ramírez Castillo,747.0,-0.063523,-0.007653
388,228902,Jonathan Calleri,3210.0,-0.275298,-0.007719
389,3875,Borja González Tomás,1029.0,-0.095660,-0.008367


In [161]:
dfs = [player_ratings['Spain'], player_ratings['England'], player_ratings['Germany'], player_ratings['Italy'], player_ratings['France']]

df_final = pd.concat(dfs)

In [218]:
transfers = pd.read_csv('data/transfers/all_transfers.csv')
all_players = pd.read_csv('data/players/all_players.csv')
recent_transfers = transfers[transfers['season']>=2017]
detailed_transfers = pd.merge(recent_transfers, all_players[['playerLink', 'currAge', 'height', 'foot']], on="playerLink", how="left")

detailed_transfers

,playerName,playerLink,playerAge,playerPos,playerCountry,playerMarketValue,transferFee,clubLeft,clubLeftUrl,clubJoined,clubJoinedUrl,teamTransferType,season,valuation,currAge,height,foot
0,Ethan Ampadu,/ethan-ampadu/profil/spieler/392771,16,CB,Wales,9000000,2800000,Exeter City,/exeter-city/startseite/verein/6699,Chelsea FC,/fc-chelsea/startseite/verein/631,in,2017,6200000,22.0,1.82,right
1,Davinson Sánchez,/davinson-sanchez/profil/spieler/341429,21,CB,Colombia,15000000,42000000,Ajax Amsterdam,/ajax-amsterdam/startseite/verein/610,Tottenham Hotspur,/tottenham-hotspur/startseite/verein/148,in,2017,-27000000,27.0,1.87,right
2,Juan Foyth,/juan-foyth/profil/spieler/480763,19,RB,Argentina,25000000,10500000,Club Estudiantes de La Plata,/club-estudiantes-de-la-plata/startseite/verei...,Tottenham Hotspur,/tottenham-hotspur/startseite/verein/148,in,2017,14500000,25.0,1.87,right
3,Kyle Walker,/kyle-walker/profil/spieler/95424,27,RB,England,13000000,52700000,Tottenham Hotspur,/tottenham-hotspur/startseite/verein/148,Manchester City,/manchester-city/startseite/verein/281,in,2017,-39700000,33.0,1.83,right
4,Ederson,/ederson/profil/spieler/238223,23,GK,Brazil,40000000,40000000,SL Benfica,/benfica-lissabon/startseite/verein/294,Manchester City,/manchester-city/startseite/verein/281,in,2017,0,29.0,1.88,left
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3986,Ibrahim Cissoko,/ibrahim-cissoko/profil/spieler/575693,20,LW,Netherlands,2500000,3000000,NEC Nijmegen,/nec-nijmegen/startseite/verein/467,FC Toulouse,/fc-toulouse/startseite/verein/415,in,2023,-500000,20.0,NaN,right
3987,Mahdi Camara,/mahdi-camara/profil/spieler/324795,25,CM,France,3000000,3000000,AS Saint-Étienne,/as-saint-etienne/startseite/verein/618,Stade Brestois 29,/stade-brest-29/startseite/verein/3911,in,2023,0,25.0,1.78,right
3988,Mostafa Mohamed,/mostafa-mohamed/profil/spieler/462348,25,CF,Egypt,6000000,5750000,Galatasaray,/galatasaray-istanbul/startseite/verein/141,FC Nantes,/fc-nantes/startseite/verein/995,in,2023,250000,25.0,1.85,right
3989,Andy Delort,/andy-delort/profil/spieler/122797,31,CF,Algeria,7000000,5000000,OGC Nice,/ogc-nizza/startseite/verein/417,FC Nantes,/fc-nantes/startseite/verein/995,in,2023,2000000,31.0,1.81,right


In [219]:
stats_types = ["standard", "shooting", "passing", "passing_types", "goal_shot_creation", "defense", "possession", "playing_time", "keeper", "misc"]

complete_transfers = {}
for type in stats_types:
    complete_transfers[type] = pd.read_csv(f'data/transfers_stats/new/all/{type}.csv')

complete_transfers['standard']
                                
complete_transfers['standard'].drop(['playerName', 'playerLink', 'playerAge', 'playerMarketValue', 'transferFee', 'clubLeftUrl', 'clubJoinedUrl', 'teamTransferType'], axis=1).iloc[:, :20]

,playerPos,playerCountry,clubLeft,clubJoined,season,valuation,currAge,height,foot,league,team,nation,pos,age,born,Playing Time_MP,Playing Time_Starts,Playing Time_Min,Playing Time_90s,Performance_Gls
0,CB,Wales,Exeter City,Chelsea FC,2017,6200000,22.0,1.82,right,ENG-Premier League,Chelsea,WAL,DF,16.0,2000.0,1,0,11,0.1,0
1,CB,Colombia,Ajax Amsterdam,Tottenham Hotspur,2017,-27000000,27.0,1.87,right,ENG-Premier League,Tottenham,COL,DF,21.0,1996.0,31,29,2534,28.2,0
2,RB,England,Tottenham Hotspur,Manchester City,2017,-39700000,33.0,1.83,right,ENG-Premier League,Manchester City,ENG,DF,27.0,1990.0,32,32,2785,30.9,0
3,GK,Brazil,SL Benfica,Manchester City,2017,0,29.0,1.88,left,ENG-Premier League,Manchester City,BRA,GK,23.0,1993.0,36,36,3194,35.5,0
4,CM,Brazil,Clube de Regatas Vasco da Gama,Manchester City,2017,28000000,25.0,1.77,right,ESP-La Liga,Girona,BRA,"MF,FW",19.0,1998.0,15,1,309,3.4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2669,LB,Sweden,IF Elfsborg,FC Toulouse,2022,-500000,21.0,1.83,NaN,FRA-Ligue 1,Toulouse,SWE,"DF,MF",20.0,2001.0,5,1,146,1.6,0
2670,RB,France,FC Toulouse,LOSC Lille,2022,7000000,22.0,1.85,right,FRA-Ligue 1,Lille,FRA,DF,21.0,2001.0,33,29,2653,29.5,3
2671,CB,Congo,Le Havre AC,AC Ajaccio,2022,300000,27.0,1.84,right,FRA-Ligue 1,Ajaccio,CGO,DF,26.0,1996.0,11,8,707,7.9,0
2672,DM,Madagascar,Los Angeles Galaxy,AJ Auxerre,2022,50000,26.0,1.83,right,FRA-Ligue 1,Auxerre,MAD,"DF,MF",25.0,1997.0,34,27,2431,27.0,2


In [220]:
detailed_transfers['lessThan20Years'] = 0
detailed_transfers['21To24Years'] = 0
detailed_transfers['26To30Years'] = 0
detailed_transfers['31To35Years'] = 0
detailed_transfers['MoreThan35Years'] = 0

# Define os valores das dummies com base na idade
detailed_transfers.loc[detailed_transfers['playerAge'] <= 20, 'lessThan20Years'] = 1
detailed_transfers.loc[(detailed_transfers['playerAge'] >= 21) & (detailed_transfers['playerAge'] <= 25), '21To24Years'] = 1
detailed_transfers.loc[(detailed_transfers['playerAge'] >= 26) & (detailed_transfers['playerAge'] <= 30), '26To30Years'] = 1
detailed_transfers.loc[(detailed_transfers['playerAge'] >= 31) & (detailed_transfers['playerAge'] <= 35), '31To35Years'] = 1
detailed_transfers.loc[detailed_transfers['playerAge'] > 35, 'MoreThan35Years'] = 1

detailed_transfers

,playerName,playerLink,playerAge,playerPos,playerCountry,playerMarketValue,transferFee,clubLeft,clubLeftUrl,clubJoined,...,season,valuation,currAge,height,foot,lessThan20Years,21To24Years,26To30Years,31To35Years,MoreThan35Years
0,Ethan Ampadu,/ethan-ampadu/profil/spieler/392771,16,CB,Wales,9000000,2800000,Exeter City,/exeter-city/startseite/verein/6699,Chelsea FC,...,2017,6200000,22.0,1.82,right,1,0,0,0,0
1,Davinson Sánchez,/davinson-sanchez/profil/spieler/341429,21,CB,Colombia,15000000,42000000,Ajax Amsterdam,/ajax-amsterdam/startseite/verein/610,Tottenham Hotspur,...,2017,-27000000,27.0,1.87,right,0,1,0,0,0
2,Juan Foyth,/juan-foyth/profil/spieler/480763,19,RB,Argentina,25000000,10500000,Club Estudiantes de La Plata,/club-estudiantes-de-la-plata/startseite/verei...,Tottenham Hotspur,...,2017,14500000,25.0,1.87,right,1,0,0,0,0
3,Kyle Walker,/kyle-walker/profil/spieler/95424,27,RB,England,13000000,52700000,Tottenham Hotspur,/tottenham-hotspur/startseite/verein/148,Manchester City,...,2017,-39700000,33.0,1.83,right,0,0,1,0,0
4,Ederson,/ederson/profil/spieler/238223,23,GK,Brazil,40000000,40000000,SL Benfica,/benfica-lissabon/startseite/verein/294,Manchester City,...,2017,0,29.0,1.88,left,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3986,Ibrahim Cissoko,/ibrahim-cissoko/profil/spieler/575693,20,LW,Netherlands,2500000,3000000,NEC Nijmegen,/nec-nijmegen/startseite/verein/467,FC Toulouse,...,2023,-500000,20.0,NaN,right,1,0,0,0,0
3987,Mahdi Camara,/mahdi-camara/profil/spieler/324795,25,CM,France,3000000,3000000,AS Saint-Étienne,/as-saint-etienne/startseite/verein/618,Stade Brestois 29,...,2023,0,25.0,1.78,right,0,1,0,0,0
3988,Mostafa Mohamed,/mostafa-mohamed/profil/spieler/462348,25,CF,Egypt,6000000,5750000,Galatasaray,/galatasaray-istanbul/startseite/verein/141,FC Nantes,...,2023,250000,25.0,1.85,right,0,1,0,0,0
3989,Andy Delort,/andy-delort/profil/spieler/122797,31,CF,Algeria,7000000,5000000,OGC Nice,/ogc-nizza/startseite/verein/417,FC Nantes,...,2023,2000000,31.0,1.81,right,0,0,0,1,0


In [221]:
anos_desejados = [2017, 2018]
detailed_transfers = detailed_transfers.loc[detailed_transfers['season'].isin(anos_desejados)]

In [222]:
detailed_transfers

,playerName,playerLink,playerAge,playerPos,playerCountry,playerMarketValue,transferFee,clubLeft,clubLeftUrl,clubJoined,...,season,valuation,currAge,height,foot,lessThan20Years,21To24Years,26To30Years,31To35Years,MoreThan35Years
0,Ethan Ampadu,/ethan-ampadu/profil/spieler/392771,16,CB,Wales,9000000,2800000,Exeter City,/exeter-city/startseite/verein/6699,Chelsea FC,...,2017,6200000,22.0,1.82,right,1,0,0,0,0
1,Davinson Sánchez,/davinson-sanchez/profil/spieler/341429,21,CB,Colombia,15000000,42000000,Ajax Amsterdam,/ajax-amsterdam/startseite/verein/610,Tottenham Hotspur,...,2017,-27000000,27.0,1.87,right,0,1,0,0,0
2,Juan Foyth,/juan-foyth/profil/spieler/480763,19,RB,Argentina,25000000,10500000,Club Estudiantes de La Plata,/club-estudiantes-de-la-plata/startseite/verei...,Tottenham Hotspur,...,2017,14500000,25.0,1.87,right,1,0,0,0,0
3,Kyle Walker,/kyle-walker/profil/spieler/95424,27,RB,England,13000000,52700000,Tottenham Hotspur,/tottenham-hotspur/startseite/verein/148,Manchester City,...,2017,-39700000,33.0,1.83,right,0,0,1,0,0
4,Ederson,/ederson/profil/spieler/238223,23,GK,Brazil,40000000,40000000,SL Benfica,/benfica-lissabon/startseite/verein/294,Manchester City,...,2017,0,29.0,1.88,left,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3381,John Bostock,/john-bostock/profil/spieler/58097,26,CM,Trinidad and Tobago,0,700000,Bursaspor,/bursaspor/startseite/verein/20,FC Toulouse,...,2018,-700000,31.0,1.88,left,0,0,1,0,0
3382,Issa Diop,/issa-diop/profil/spieler/272622,21,CB,France,18000000,25000000,FC Toulouse,/fc-toulouse/startseite/verein/415,West Ham United,...,2018,-7000000,26.0,1.94,right,0,1,0,0,0
3383,Alban Lafont,/alban-lafont/profil/spieler/357117,19,GK,France,16000000,8000000,FC Toulouse,/fc-toulouse/startseite/verein/415,ACF Fiorentina,...,2018,8000000,24.0,1.96,right,1,0,0,0,0
3384,Somália,/somalia/profil/spieler/148754,29,CM,Brazil,200000,1500000,FC Toulouse,/fc-toulouse/startseite/verein/415,Al-Shabab FC,...,2018,-1300000,34.0,1.81,both,0,0,1,0,0


In [223]:
df_final

,player_id,playerName,minutes_played,vaep_total,vaep_p90
0,3359,Lionel Andrés Messi Cuccittini,3108.0,35.891377,1.039326
1,8278,Gareth Frank Bale,1850.0,14.323647,0.696826
2,3802,Philippe Coutinho Correia,1329.0,10.036555,0.679676
3,3322,Cristiano Ronaldo dos Santos Aveiro,2355.0,17.279105,0.660348
4,3682,Antoine Griezmann,2591.0,18.149570,0.630437
...,...,...,...,...,...
380,134542,Nampalys Mendy,994.0,0.089895,0.008139
381,234138,Jan Repas,931.0,0.050414,0.004874
382,101795,Lacina Traoré,826.0,-0.317548,-0.034600
383,123,Ola Toivonen,888.0,-0.372171,-0.037720


In [224]:
%pip install pycountry_convert

Note: you may need to restart the kernel to use updated packages.


In [225]:
import pycountry_convert as pc

def get_continent(country_name):
    try:
        country_alpha2 = pc.country_name_to_country_alpha2(country_name)
        continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
        continent_name = pc.convert_continent_code_to_continent_name(continent_code)
        return continent_name
    # adding manual continents for some countries
    except KeyError:
        if country_name in ['Wales', 'England', 'Scotland', 'Bosnia-Herzegovina', 'Northern Ireland', 'Kosovo']:
            return 'Europe'
        elif country_name in ["Cote d'Ivoire" ,'The Gambia', 'Curacao', 'DR Congo' ]:
            return 'Africa'
        elif country_name in ['Korea, South', 'Korea, North']:
            return 'Asia'
        elif country_name in ['Neukaledonien']:
            return 'Oceania'
        else:
            return None
    

detailed_transfers['continent'] = detailed_transfers['playerCountry'].apply(get_continent)
detailed_transfers['european'] = 0
detailed_transfers['asian'] = 0
detailed_transfers['southAmerican'] = 0
detailed_transfers['african'] = 0
detailed_transfers['northAmerican'] = 0
detailed_transfers['oceanian'] = 0

detailed_transfers.loc[detailed_transfers['continent'] == 'Europe', 'european'] = 1
detailed_transfers.loc[detailed_transfers['continent'] == 'Asia', 'asian'] = 1
detailed_transfers.loc[detailed_transfers['continent'] == 'South America', 'southAmerican'] = 1
detailed_transfers.loc[detailed_transfers['continent'] == 'North America', 'northAmerican'] = 1
detailed_transfers.loc[detailed_transfers['continent'] == 'Oceania', 'oceanian'] = 1

detailed_transfers

C:\Users\Leonardo\AppData\Local\Temp\ipykernel_6720\3155819422.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  detailed_transfers['continent'] = detailed_transfers['playerCountry'].apply(get_continent)
C:\Users\Leonardo\AppData\Local\Temp\ipykernel_6720\3155819422.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  detailed_transfers['european'] = 0
C:\Users\Leonardo\AppData\Local\Temp\ipykernel_6720\3155819422.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

,playerName,playerLink,playerAge,playerPos,playerCountry,playerMarketValue,transferFee,clubLeft,clubLeftUrl,clubJoined,...,26To30Years,31To35Years,MoreThan35Years,continent,european,asian,southAmerican,african,northAmerican,oceanian
0,Ethan Ampadu,/ethan-ampadu/profil/spieler/392771,16,CB,Wales,9000000,2800000,Exeter City,/exeter-city/startseite/verein/6699,Chelsea FC,...,0,0,0,Europe,1,0,0,0,0,0
1,Davinson Sánchez,/davinson-sanchez/profil/spieler/341429,21,CB,Colombia,15000000,42000000,Ajax Amsterdam,/ajax-amsterdam/startseite/verein/610,Tottenham Hotspur,...,0,0,0,South America,0,0,1,0,0,0
2,Juan Foyth,/juan-foyth/profil/spieler/480763,19,RB,Argentina,25000000,10500000,Club Estudiantes de La Plata,/club-estudiantes-de-la-plata/startseite/verei...,Tottenham Hotspur,...,0,0,0,South America,0,0,1,0,0,0
3,Kyle Walker,/kyle-walker/profil/spieler/95424,27,RB,England,13000000,52700000,Tottenham Hotspur,/tottenham-hotspur/startseite/verein/148,Manchester City,...,1,0,0,Europe,1,0,0,0,0,0
4,Ederson,/ederson/profil/spieler/238223,23,GK,Brazil,40000000,40000000,SL Benfica,/benfica-lissabon/startseite/verein/294,Manchester City,...,0,0,0,South America,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3381,John Bostock,/john-bostock/profil/spieler/58097,26,CM,Trinidad and Tobago,0,700000,Bursaspor,/bursaspor/startseite/verein/20,FC Toulouse,...,1,0,0,North America,0,0,0,0,1,0
3382,Issa Diop,/issa-diop/profil/spieler/272622,21,CB,France,18000000,25000000,FC Toulouse,/fc-toulouse/startseite/verein/415,West Ham United,...,0,0,0,Europe,1,0,0,0,0,0
3383,Alban Lafont,/alban-lafont/profil/spieler/357117,19,GK,France,16000000,8000000,FC Toulouse,/fc-toulouse/startseite/verein/415,ACF Fiorentina,...,0,0,0,Europe,1,0,0,0,0,0
3384,Somália,/somalia/profil/spieler/148754,29,CM,Brazil,200000,1500000,FC Toulouse,/fc-toulouse/startseite/verein/415,Al-Shabab FC,...,1,0,0,South America,0,0,1,0,0,0


In [226]:
#for season in detailed_transfers['season'].unique():
 #   mean = detailed_transfers.loc[detailed_transfers['season'] == season, 'transferFee'].mean()
  #  detailed_transfers.loc[detailed_transfers['season'] == season, 'transferFee'] /= mean

#sorted_df = detailed_transfers.sort_values('transferFee', ascending=False)
#sorted_df

In [227]:
import numpy as np

height_percentiles = np.nanpercentile(detailed_transfers['height'], [25, 75])

# Assign category labels based on the percentiles
detailed_transfers['tall'] = 0
detailed_transfers['short'] = 0
detailed_transfers['average'] = 0

detailed_transfers.loc[detailed_transfers['height'] <= height_percentiles[0],'short'] = 1
detailed_transfers.loc[detailed_transfers['height'] >= height_percentiles[1],'tall'] = 1
detailed_transfers.loc[(detailed_transfers['height'] > height_percentiles[0]) & (detailed_transfers['height'] < height_percentiles[1]), 'average'] = 1


print(detailed_transfers['height'])
print(height_percentiles)
detailed_transfers[detailed_transfers['playerName'] == 'Cristiano Ronaldo']


0       1.82
1       1.87
2       1.87
3       1.83
4       1.88
        ... 
3381    1.88
3382    1.94
3383    1.96
3384    1.81
3385    1.90
Name: height, Length: 1433, dtype: float64
[1.78 1.87]


C:\Users\Leonardo\AppData\Local\Temp\ipykernel_6720\3751969880.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  detailed_transfers['tall'] = 0
C:\Users\Leonardo\AppData\Local\Temp\ipykernel_6720\3751969880.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  detailed_transfers['short'] = 0
C:\Users\Leonardo\AppData\Local\Temp\ipykernel_6720\3751969880.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

,playerName,playerLink,playerAge,playerPos,playerCountry,playerMarketValue,transferFee,clubLeft,clubLeftUrl,clubJoined,...,continent,european,asian,southAmerican,african,northAmerican,oceanian,tall,short,average
2209,Cristiano Ronaldo,/cristiano-ronaldo/profil/spieler/8198,33,CF,Portugal,15000000,117000000,Real Madrid,/real-madrid/startseite/verein/418,Juventus FC,...,Europe,1,0,0,0,0,0,1,0,0


In [228]:
detailed_transfers['ambidextrous'] = 0
detailed_transfers.loc[detailed_transfers['foot'] == 'both', 'ambidextrous'] = 1

detailed_transfers

C:\Users\Leonardo\AppData\Local\Temp\ipykernel_6720\3542202882.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  detailed_transfers['ambidextrous'] = 0


,playerName,playerLink,playerAge,playerPos,playerCountry,playerMarketValue,transferFee,clubLeft,clubLeftUrl,clubJoined,...,european,asian,southAmerican,african,northAmerican,oceanian,tall,short,average,ambidextrous
0,Ethan Ampadu,/ethan-ampadu/profil/spieler/392771,16,CB,Wales,9000000,2800000,Exeter City,/exeter-city/startseite/verein/6699,Chelsea FC,...,1,0,0,0,0,0,0,0,1,0
1,Davinson Sánchez,/davinson-sanchez/profil/spieler/341429,21,CB,Colombia,15000000,42000000,Ajax Amsterdam,/ajax-amsterdam/startseite/verein/610,Tottenham Hotspur,...,0,0,1,0,0,0,1,0,0,0
2,Juan Foyth,/juan-foyth/profil/spieler/480763,19,RB,Argentina,25000000,10500000,Club Estudiantes de La Plata,/club-estudiantes-de-la-plata/startseite/verei...,Tottenham Hotspur,...,0,0,1,0,0,0,1,0,0,0
3,Kyle Walker,/kyle-walker/profil/spieler/95424,27,RB,England,13000000,52700000,Tottenham Hotspur,/tottenham-hotspur/startseite/verein/148,Manchester City,...,1,0,0,0,0,0,0,0,1,0
4,Ederson,/ederson/profil/spieler/238223,23,GK,Brazil,40000000,40000000,SL Benfica,/benfica-lissabon/startseite/verein/294,Manchester City,...,0,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3381,John Bostock,/john-bostock/profil/spieler/58097,26,CM,Trinidad and Tobago,0,700000,Bursaspor,/bursaspor/startseite/verein/20,FC Toulouse,...,0,0,0,0,1,0,1,0,0,0
3382,Issa Diop,/issa-diop/profil/spieler/272622,21,CB,France,18000000,25000000,FC Toulouse,/fc-toulouse/startseite/verein/415,West Ham United,...,1,0,0,0,0,0,1,0,0,0
3383,Alban Lafont,/alban-lafont/profil/spieler/357117,19,GK,France,16000000,8000000,FC Toulouse,/fc-toulouse/startseite/verein/415,ACF Fiorentina,...,1,0,0,0,0,0,1,0,0,0
3384,Somália,/somalia/profil/spieler/148754,29,CM,Brazil,200000,1500000,FC Toulouse,/fc-toulouse/startseite/verein/415,Al-Shabab FC,...,0,0,1,0,0,0,0,0,1,1


In [229]:
pos_mapping = {
    'RM': 'RW',
    'LM': 'LW',
    'SS': 'AM'
}

In [230]:
detailed_transfers['playerPos'] = detailed_transfers['playerPos'].replace(pos_mapping)

C:\Users\Leonardo\AppData\Local\Temp\ipykernel_6720\2491225570.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  detailed_transfers['playerPos'] = detailed_transfers['playerPos'].replace(pos_mapping)


In [231]:
player_ratings['Spain'].columns

Index(['player_id', 'player_name', 'minutes_played', 'vaep_total', 'vaep_p90'], dtype='object')

In [232]:
df_final = df_final.rename(columns={'player_name': 'playerName'})

In [233]:
detailed_transfers.columns

Index(['playerName', 'playerLink', 'playerAge', 'playerPos', 'playerCountry',
       'playerMarketValue', 'transferFee', 'clubLeft', 'clubLeftUrl',
       'clubJoined', 'clubJoinedUrl', 'teamTransferType', 'season',
       'valuation', 'currAge', 'height', 'foot', 'lessThan20Years',
       '21To24Years', '26To30Years', '31To35Years', 'MoreThan35Years',
       'continent', 'european', 'asian', 'southAmerican', 'african',
       'northAmerican', 'oceanian', 'tall', 'short', 'average',
       'ambidextrous'],
      dtype='object')

In [234]:
coluna_em_comum = 'playerName'


#df_final = pd.merge(detailed_transfers, player_ratings, left_on=coluna_em_comum, right_on='player_name')
df_junto = pd.merge(detailed_transfers, df_final, on=['playerName'])

In [235]:
df_junto

,playerName,playerLink,playerAge,playerPos,playerCountry,playerMarketValue,transferFee,clubLeft,clubLeftUrl,clubJoined,...,northAmerican,oceanian,tall,short,average,ambidextrous,player_id,minutes_played,vaep_total,vaep_p90
0,Kyle Walker,/kyle-walker/profil/spieler/95424,27,RB,England,13000000,52700000,Tottenham Hotspur,/tottenham-hotspur/startseite/verein/148,Manchester City,...,0,0,0,0,1,0,8277,2902.0,6.692535,0.207556
1,Andrew Robertson,/andrew-robertson/profil/spieler/234803,23,LB,Scotland,40000000,9000000,Hull City,/hull-city/startseite/verein/3008,Liverpool FC,...,0,0,0,1,0,0,222220,2012.0,4.987659,0.223106
2,Alex Oxlade-Chamberlain,/alex-oxlade-chamberlain/profil/spieler/143424,24,CM,England,7000000,38000000,Arsenal FC,/fc-arsenal/startseite/verein/11,Liverpool FC,...,0,0,0,1,0,0,7868,1816.0,7.746565,0.383916
3,Olivier Giroud,/olivier-giroud/profil/spieler/82442,31,CF,France,4000000,17000000,Arsenal FC,/fc-arsenal/startseite/verein/11,Chelsea FC,...,0,0,1,0,0,0,26010,1093.0,4.474742,0.368460
4,Henrikh Mkhitaryan,/henrikh-mkhitaryan/profil/spieler/55735,29,AM,Armenia,6000000,34000000,Manchester United,/manchester-united/startseite/verein/985,Arsenal FC,...,0,0,0,1,0,1,105338,1762.0,4.259215,0.217554
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,Lebo Mothiba,/lebo-mothiba/profil/spieler/427161,22,CF,South Africa,1200000,4000000,LOSC Lille,/losc-lille/startseite/verein/1082,RC Strasbourg Alsace,...,0,0,0,0,1,0,366546,1216.0,3.793878,0.280797
377,Baptiste Reynet,/baptiste-reynet/profil/spieler/190786,27,GK,France,600000,4000000,Dijon FCO,/dijon-fco/startseite/verein/2969,FC Toulouse,...,0,0,0,0,1,0,26365,3541.0,5.667710,0.144054
378,Mathieu Dossevi,/mathieu-dossevi/profil/spieler/77828,30,RW,Togo,100000,2500000,FC Metz,/fc-metz/startseite/verein/347,FC Toulouse,...,0,0,0,1,0,0,26512,2585.0,5.276081,0.183693
379,Issa Diop,/issa-diop/profil/spieler/272622,21,CB,France,18000000,25000000,FC Toulouse,/fc-toulouse/startseite/verein/415,West Ham United,...,0,0,1,0,0,0,286380,3195.0,5.868491,0.165310


In [236]:
df_junto.columns

Index(['playerName', 'playerLink', 'playerAge', 'playerPos', 'playerCountry',
       'playerMarketValue', 'transferFee', 'clubLeft', 'clubLeftUrl',
       'clubJoined', 'clubJoinedUrl', 'teamTransferType', 'season',
       'valuation', 'currAge', 'height', 'foot', 'lessThan20Years',
       '21To24Years', '26To30Years', '31To35Years', 'MoreThan35Years',
       'continent', 'european', 'asian', 'southAmerican', 'african',
       'northAmerican', 'oceanian', 'tall', 'short', 'average', 'ambidextrous',
       'player_id', 'minutes_played', 'vaep_total', 'vaep_p90'],
      dtype='object')

In [237]:
df_junto2 = df_junto.drop(['playerLink', 'playerMarketValue', 'clubLeft', 'clubLeftUrl',
       'clubJoined', 'clubJoinedUrl', 'teamTransferType', 'season',
       'valuation', 'currAge', 'player_id', 'minutes_played' ], axis = 1)

In [238]:
correlacao = df_junto2.corr()['transferFee']

C:\Users\Leonardo\AppData\Local\Temp\ipykernel_6720\361319624.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlacao = df_junto2.corr()['transferFee']


In [239]:
correlacao

playerAge         -0.123707
transferFee        1.000000
height             0.017566
lessThan20Years    0.089754
21To24Years       -0.001274
26To30Years        0.020273
31To35Years       -0.121216
MoreThan35Years   -0.033426
european          -0.007709
asian              0.067043
southAmerican     -0.062938
african                 NaN
northAmerican      0.075141
oceanian          -0.024047
tall               0.006945
short              0.047212
average           -0.047298
ambidextrous       0.174635
vaep_total         0.221700
vaep_p90           0.245172
Name: transferFee, dtype: float64